# PyTorch Developer's Guide to JAX Fundamentals

This notebook along with its accompanying blog post serves to help PyTorch users get familiar with Jax/Flax by connecting the building blocks they are already familiar with in PyTorch to their equivalents in Jax/Flax!

### Data Exploration
We will be making use of the classic Titanic dataset. See
https://www.kaggle.com/c/titanic/ for details.

In [2]:
import pandas as pd

df = pd.read_csv("data/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def preprocess_titanic_data(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.Series]:
    """
    Basic preprocessing of the Titanic dataset for usage within ML training
    """
    df = df.drop(columns=["Name", "Ticket", "Cabin"], axis=1)
    df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
    df["Embarked"] = df["Embarked"].map({"S": 0, "C": 1, "Q": 2})
    df["Age"] = df["Age"].fillna(df["Age"].median())
    df = df.dropna()
    labels = df.pop("Survived")
    return df, labels


df, labels = preprocess_titanic_data(df)
df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,0,22.0,1,0,7.2500,0.0
1,2,1,1,38.0,1,0,71.2833,1.0
2,3,3,1,26.0,0,0,7.9250,0.0
3,4,1,1,35.0,1,0,53.1000,0.0
4,5,3,0,35.0,0,0,8.0500,0.0


In [4]:
# Split data into train and test/eval
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df, labels, test_size=0.2, random_state=42
)

As a baseline, let's train a simple RandomForestClassifier with this data

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.797752808988764


### Dataloader
See https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html for more details

In [6]:
import torch
import jax.numpy as jnp
from torch.utils.data import Dataset, DataLoader, default_collate
from jax.tree_util import tree_map


class TitanicDataset(Dataset):
    def __init__(self, samples: pd.DataFrame, labels: pd.Series):
        self.df = samples
        self.labels = labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(self.labels.iloc[idx], dtype=torch.int32)
        return x, y


# This collate function is taken from the JAX tutorial with PyTorch Data Loading
# https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html
def numpy_collate(batch):
    return tree_map(jnp.asarray, default_collate(batch))


train_dataset = TitanicDataset(X_train, y_train)
train_dataloader_jax = DataLoader(
    train_dataset, batch_size=64, shuffle=True, collate_fn=numpy_collate
)
train_dataloader_torch = DataLoader(train_dataset, batch_size=64, shuffle=True)

eval_dataset = TitanicDataset(X_test, y_test)
eval_dataloader_jax = DataLoader(
    eval_dataset, batch_size=64, shuffle=False, collate_fn=numpy_collate
)
eval_dataloader_torch = DataLoader(eval_dataset, batch_size=64, shuffle=False)

### PyTorch Reference Implementation and Training

Model Definition

In [7]:
import torch.nn as nn
import torch.optim as optim


class TitanicNeuralNet(nn.Module):
    def __init__(self, num_hidden_1, num_hidden_2):
        super().__init__()
        self.linear1 = nn.Linear(8, num_hidden_1)
        self.dropout = nn.Dropout(0.01)
        self.relu = nn.LeakyReLU()
        self.linear2 = nn.Linear(num_hidden_1, num_hidden_2)
        self.linear3 = nn.Linear(num_hidden_2, 1, bias=False)

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.relu(x)
        out = self.linear3(x)
        return out

Training Loop

In [8]:
from tqdm import tqdm


def train(
    model: nn.Module,
    train_dataloader: DataLoader,
    eval_dataloader: DataLoader,
    num_epochs: int,
):
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.BCEWithLogitsLoss()
    for epoch in (pbar := tqdm(range(num_epochs))):
        pbar.set_description(f"Epoch {epoch}")
        model.train()
        for batch, labels in train_dataloader:
            # zero your gradients
            optimizer.zero_grad()

            # forward pass
            logits = model(batch)

            # compute loss
            loss = criterion(logits.squeeze(), labels.float())

            # backward pass
            loss.backward()

            # take an optimizer step
            optimizer.step()

        pbar.set_postfix(
            train_accuracy=eval(model, train_dataloader),
            eval_accuracy=eval(model, eval_dataloader),
        )


def eval(model: nn.Module, eval_dataloader: DataLoader):
    model.eval()
    num_correct = 0
    num_samples = 0
    for batch, labels in eval_dataloader:
        logits = model(batch)
        preds = torch.round(torch.sigmoid(logits))
        num_correct += (preds.squeeze() == labels).sum().item()
        num_samples += labels.shape[0]
    return num_correct / num_samples


model = TitanicNeuralNet(num_hidden_1=32, num_hidden_2=16)
train(model, train_dataloader_torch, eval_dataloader_torch, num_epochs=500)

Epoch 499: 100%|██████████| 500/500 [00:29<00:00, 16.77it/s, eval_accuracy=0.787, train_accuracy=0.858]


### Flax NNX (Object-Oriented) Neural Net
See https://flax.readthedocs.io/en/latest/index.html for full NNX documentation, with more examples and a deeper dive.

Model Definition

In [17]:
from flax import nnx


class TitanicNNX(nnx.Module):
    def __init__(self, num_hidden_1: int, num_hidden_2: int, rngs: nnx.Rngs):
        self.linear1 = nnx.Linear(8, num_hidden_1, rngs=rngs)
        self.dropout = nnx.Dropout(0.01, rngs=rngs)
        self.relu = nnx.leaky_relu
        self.linear2 = nnx.Linear(num_hidden_1, num_hidden_2, rngs=rngs)
        self.linear3 = nnx.Linear(num_hidden_2, 1, use_bias=False, rngs=rngs)

    def __call__(self, x):
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.relu(x)
        out = self.linear3(x)
        return out

Initialize the Model

In [18]:
model = TitanicNNX(32, 16, rngs=nnx.Rngs(0))
nnx.display(model)

TitanicNNX(
  linear1=Linear(
    kernel=Param(
      value=Array(shape=(8, 32), dtype=float32)
    ),
    bias=Param(
      value=Array(shape=(32,), dtype=float32)
    ),
    in_features=8,
    out_features=32,
    use_bias=True,
    dtype=None,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    kernel_init=<function variance_scaling.<locals>.init at 0x7fb7b81808b0>,
    bias_init=<function zeros at 0x7fba9b00b9a0>,
    dot_general=<function dot_general at 0x7fba9ba1dbd0>
  ),
  dropout=Dropout(rate=0.01, broadcast_dims=(), deterministic=False, rng_collection='dropout', rngs=Rngs(
    default=RngStream(
      key=RngKey(
        value=Array((), dtype=key<fry>) overlaying:
        [0 0],
        tag='default'
      ),
      count=RngCount(
        value=Array(5, dtype=uint32),
        tag='default'
      )
    )
  )),
  relu=<PjitFunction of <function leaky_relu at 0x7fba9b0a2d40>>,
  linear2=Linear(
    kernel=Param(
      value=Array(shape=(32, 16), dtype=float32)
 

Training Loop Setup

In [19]:
import optax
from jax import Array


def train(
    model: nnx.Module,
    train_dataloader: DataLoader,
    eval_dataloader: DataLoader,
    num_epochs=int,
):
    optimizer = nnx.Optimizer(model, optax.adam(learning_rate=0.01))

    for epoch in (pbar := tqdm(range(num_epochs))):
        pbar.set_description(f"Epoch {epoch}")
        model.train()
        for batch in train_dataloader:
            train_step(model, optimizer, batch)

        pbar.set_postfix(
            train_accuracy=eval(model, train_dataloader),
            eval_accuracy=eval(model, eval_dataloader),
        )


@nnx.jit
def train_step(model: nnx.Module, optimizer: nnx.Optimizer, batch: Array):
    def loss_fn(model):
        logits = model(batch[0])
        loss = optax.sigmoid_binary_cross_entropy(logits.squeeze(), batch[1]).mean()
        return loss

    grad_fn = nnx.value_and_grad(loss_fn)
    loss, grads = grad_fn(model)
    optimizer.update(grads)


def eval(model: nnx.Module, eval_dataloader: DataLoader):
    total = 0
    num_correct = 0
    model.eval()
    for batch in eval_dataloader:
        res = eval_step(model, batch)
        total += res.shape[0]
        num_correct += jnp.sum(res)
    return num_correct / total


@nnx.jit
def eval_step(model: nnx.Module, batch: Array):
    logits = model(batch[0])
    logits = logits.squeeze()
    preds = jnp.round(nnx.sigmoid(logits))
    return preds == batch[1]

Initiate Training

In [20]:
train(model, train_dataloader_jax, eval_dataloader_jax, num_epochs=500)

Epoch 499: 100%|██████████| 500/500 [00:38<00:00, 13.03it/s, eval_accuracy=0.76404494, train_accuracy=0.84388185]


### Flax Linen (Functional) Neural Net

See https://flax-linen.readthedocs.io/en/latest/index.html for full Linen documentation, with more examples and a deeper dive.

In [24]:
import jax
from flax import linen as nn

initializer = jax.nn.initializers.lecun_normal()

# Model definition


# setup(approach)
class TitanicLinen(nn.Module):
    num_hidden_1: int
    num_hidden_2: int

    def setup(self):
        self.linear1 = nn.Dense(features=self.num_hidden_1, kernel_init=initializer)
        self.linear2 = nn.Dense(features=self.num_hidden_2, kernel_init=initializer)
        self.linear3 = nn.Dense(features=1, use_bias=False, kernel_init=initializer)
        self.dropout1 = nn.Dropout(0.01)
        self.dropout2 = nn.Dropout(0.01)

    def __call__(self, x, training):
        x = self.linear1(x)
        x = self.dropout1(x, deterministic=not training)
        x = nn.leaky_relu(x)
        x = self.linear2(x)
        x = self.dropout2(x, deterministic=not training)
        x = nn.leaky_relu(x)
        out = self.linear3(x)
        return out


## nn.compact approach
class TitanicLinenCompact(nn.Module):
    num_hidden_1: int
    num_hidden_2: int

    @nn.compact
    def __call__(self, x, training):
        x = nn.Dense(features=self.num_hidden_1, kernel_init=initializer)(x)
        x = nn.Dropout(0.01, deterministic=not training)(x)
        x = nn.leaky_relu(x)
        x = nn.Dense(features=self.num_hidden_2, kernel_init=initializer)(x)
        x = nn.Dropout(0.01, deterministic=not training)(x)
        x = nn.leaky_relu(x)
        out = nn.Dense(features=1, use_bias=False, kernel_init=initializer)(x)
        return out

In [25]:
rng = jax.random.PRNGKey(42)
new_rng, subkey, subdropout = jax.random.split(rng, num=3)

sample_data, sample_labels = next(iter(train_dataloader_jax))
model = TitanicLinenCompact(num_hidden_1=32, num_hidden_2=16)
params = model.init(subkey, sample_data, True)
logits = model.apply(params, sample_data, True, rngs={"dropout": subdropout})
logits.shape

(64, 1)

In [26]:
import optax
from flax.training import train_state

# Setup for train loop
optimizer = optax.adam(learning_rate=0.01)

state = train_state.TrainState.create(
    apply_fn=model.apply,
    params=params,
    tx=optimizer,
)

In [27]:
from jax import jit


# Train loop
def train(state, train_dataloader, eval_dataloader, subdropout, num_epochs):
    for epoch in (pbar := tqdm(range(num_epochs))):
        pbar.set_description(f"Epoch {epoch}")
        for batch in train_dataloader:
            state, loss = train_step(state, batch, subdropout)

        pbar.set_postfix(
            train_accuracy=eval(state, train_dataloader),
            eval_accuracy=eval(state, eval_dataloader),
        )

    return state


def eval(state, eval_dataloader):
    total = 0
    num_correct = 0
    for batch in eval_dataloader:
        res = eval_step(state, batch)
        total += res.shape[0]
        num_correct += jnp.sum(res)
    return num_correct / total


@jit
def train_step(state, batch, subdropout):
    def loss_fn(params):
        logits = state.apply_fn(params, batch[0], True, rngs={"dropout": subdropout})
        loss = optax.sigmoid_binary_cross_entropy(logits.squeeze(), batch[1]).mean()
        return loss

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)
    return state, loss


@jit
def eval_step(state, batch):
    logits = state.apply_fn(state.params, batch[0], False)
    logits = logits.squeeze()
    preds = jnp.round(nn.sigmoid(logits))
    return preds == batch[1]

In [28]:
state = train(
    state, train_dataloader_jax, eval_dataloader_jax, subdropout, num_epochs=500
)

Epoch 499: 100%|██████████| 500/500 [00:30<00:00, 16.22it/s, eval_accuracy=0.75842696, train_accuracy=0.8410689] 
